In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import itertools


import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder, PolynomialFeatures
from sklearn.utils.class_weight import compute_sample_weight


from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

import cartopy
import numpy as np
from cartopy.crs import PlateCarree
import requests
import json
from sklearn.preprocessing import OneHotEncoder

from sklearn.cluster import KMeans


import warnings
warnings.filterwarnings('ignore')


In [ ]:
def get_prov_list():
    ACCESS_KEY = '1a2febdd5530824a8bbc342c32aee3db'
    provinces = [
        'Eastern Cape',
        'Free State',
        'Gauteng',
        'KwaZulu-Natal',
        'Limpopo',
        'Mpumalanga',
        'Northern Cape',
        'North West',
        'Western Cape',
    ]   
    prov_list=[]
    for province in provinces:
        prov_dict={}
        url = 'http://api.positionstack.com/v1/forward?access_key=' + ACCESS_KEY+'&query='+province
        response = requests.get(url)
        prov_dict['province']=province
        prov_dict['lon']=response.json()['data'][0]['longitude']
        prov_dict['lat']=response.json()['data'][0]['latitude']
        prov_list.append(prov_dict)
    
    return prov_list

prov_list=get_prov_list()

In [49]:
from math import radians
from numpy import cos, sin, arcsin, sqrt
import haversine as hs
import requests
import json

# coordinates of major cities in Africa
cities_data = {
    'city': ['cape_town', 'Durban', 'Johannesburg', 'Soweto', 'Pretoria', 'Port_Elizabeth', 'PieterMaritzburg', 'Benoni', 'Tembisa', 'East_London', 'Vereeniging', 'Bloemfontein'],
    'lon': [18.423, 31.029, 28.044, 27.858, 28.188, 25.615, 30.393, 28.321, 28.227, 27.912, 27.926, 26.214],
    'lat': [-33.926, -29.858, -26.202, -26.268, -25.745, -33.961, -29.617, -26.188, -25.996, -33.015, -26.673, -29.121]
}


def haversine(row, city_index):
    lon1 = prov_list[city_index]['lon']
    lat1 = prov_list[city_index]['lat']
    lon2 = row['lon']
    lat2 = row['lat']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a))
    km = 6367 * c
    return km


def get_top10_target(df: pd.DataFrame, feature: str):
    top10_list = list(df.groupby(feature)['target'].sum(
    ).sort_values(ascending=False).head(10).index)
    return top10_list


def target_profiling(df: pd.DataFrame, feature: str):
    top_10list = get_top10_target(df, feature)
    df.loc[:, 'target.' +
           feature] = np.where(df.loc[:, feature].isin(top_10list), 1, 0)
    return df


def process(filename):
    train_df = pd.read_csv('data/Train.csv')
    test_df = pd.read_csv('data/Test.csv')
    train_df['train'] = 1
    test_df['train'] = 0
    combined = pd.concat([train_df, test_df])
    combined = combined.drop(columns=['car_01', 'stv_01', 'lln_01'])
    combined = combined.drop(columns=['ADM4_PCODE'])


    combined['dist_1'] = combined.apply(lambda row: haversine(row, 4), axis=1)
    combined['dist_2'] = combined.apply(lambda row: haversine(row, 2), axis=1)


    combined['Household_Size'] = combined['total_individuals'] / combined['total_households']


    if filename == 'data/Train.csv':
        return combined[combined.train == 1].drop(columns=['train'])
    elif filename == 'data/Test.csv':
        return combined[combined.train == 0].drop(columns=['train', 'target'])

""" with open('data/mdb_data.json') as json_file:
        mdb_data = json.load(json_file)
        mdb_df = pd.DataFrame.from_dict(pd.json_normalize(
            mdb_data['features']), orient='columns')
        mdb_df.drop(columns=['attributes.OBJECTID',
                    'geometry.rings'], inplace=True)
        combined = combined.join(mdb_df, lsuffix='ward', rsuffix='attributes.WardID').rename(columns=lambda x: x.replace("attributes.", "")).drop(columns=[
            'WardNumber', 'WardID', 'ProvinceCode', 'LocalMunicipalityCode', 'DistrictMunicipalityCode', 'Year'])
    combined['Shape__Area'] = combined['Shape__Area'].fillna(0)
    combined['Shape__Length'] = combined['Shape__Length'].fillna(0)
    combined['ProvinceName'] = combined['ProvinceName'].fillna('unknown')


    
    combined = combined.drop(columns=['ProvinceName', 'LocalMunicipalityName', 'DistrictMunicipalityName'])
    combined = combined.drop(columns=['dw_12', 'dw_13', 'lan_13', 'pw_07', 'pw_08'])

    no_labels = combined.drop(columns=['ward','target'])
    labels = combined[['ward']]
    kmeans = KMeans(n_clusters=4)
    kmeans.fit(no_labels)"""

"""combined['cluster'] = pd.Series(kmeans.labels_, index=combined.index)

    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(
        combined[['cluster']]).toarray())
    encoder_df.columns = encoder.get_feature_names(['cluster'])

    combined = combined.join(encoder_df)
    combined = combined.drop(columns=['cluster'])"""


   


"combined['cluster'] = pd.Series(kmeans.labels_, index=combined.index)\n\n    encoder = OneHotEncoder(handle_unknown='ignore')\n    encoder_df = pd.DataFrame(encoder.fit_transform(\n        combined[['cluster']]).toarray())\n    encoder_df.columns = encoder.get_feature_names(['cluster'])\n\n    combined = combined.join(encoder_df)\n    combined = combined.drop(columns=['cluster'])"

In [50]:
from cartopy.crs import LambertConformal, Mercator, Orthographic
import shapely
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader


train_data = process('data/Train.csv')
test_data = process('data/Test.csv')


fname = 'data/shapefile/gadm40_ZAF_1.shp'
shapes = list(shpreader.Reader(fname).geometries())



# coordinates of major cities in Africa
cities_data = {
    'city': ['ref', 'Stellenbosch', 'cape_town', 'Durban', 'Johannesburg', 'Soweto', 'Pretoria', 'Port_Elizabeth', 'PieterMaritzburg', 'Benoni', 'Tembisa', 'East_London', 'Vereeniging', 'Bloemfontein'],
    'lon': [30.400, 18.860151, 18.423, 31.029, 28.044, 27.858, 28.188, 25.615, 30.393, 28.321, 28.227, 27.912, 27.926, 26.214],
    'lat': [-24.202, -33.932106, -33.926, -29.858, -26.202, -26.268, -25.745, -33.961, -29.617, -26.188, -25.996, -33.015, -26.673, -29.121]
}


def add_map_features():
    ax.coastlines()
    ax.add_feature(cartopy.feature.BORDERS)
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAND)
    ax.add_geometries(shapes,PlateCarree(),edgecolor='black',facecolor='white')


plt.figure(figsize=(24, 16))
central_lon, central_lat = -10, 45
extent = [-40, 20, 30, 60]
ax = plt.axes(projection=PlateCarree())

add_map_features()

ax.scatter(train_data['lon'], train_data['lat'], c=train_data['NL'],
           transform=PlateCarree(),
           s=20,zorder=4
           )

"""ax.scatter(test_data['lon'], test_data['lat'], color='blue',# c=data['target'],
           transform=PlateCarree(),
           s=20,zorder=4
           )"""


"""for i in range(12):
    ax.text(cities_data['lon'][i], cities_data['lat'][i],
            cities_data['city'][i], transform=PlateCarree())
    ax.scatter(cities_data['lon'][i], cities_data['lat']
               [i], color='red', transform=PlateCarree(), s=30,zorder=4)"""

"""for i in range(len(prov_list)):
    ax.text(prov_list[i]['lon'], prov_list[i]['lat'],prov_list[i]['province'], transform=PlateCarree(),fontsize=20,zorder=4)
    ax.scatter(prov_list[i]['lon'], prov_list[i]['lat'], color='red', transform=PlateCarree(), s=100,zorder=4)"""

#ax.tissot(rad_km=250, lons=cities_data['lon'][0],
 #       lats=cities_data['lat'][0], alpha=0.4, color='red')



plt.show()


<IPython.core.display.Javascript object>

In [28]:
from time import time
import pprint
from sklearn.metrics import make_scorer
import joblib
from functools import partial
def report_perf(optimizer, X, y, title="model", callbacks=None):
    start = time()
    
    if callbacks is not None:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
        
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           + u"\u00B1"+" %.3f") % (time() - start, 
                                   len(optimizer.cv_results_['params']),
                                   best_score,
                                   best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params
scoring = make_scorer(partial(mean_squared_error, squared=False), 
                      greater_is_better=False)

In [51]:
X = process('data/Train.csv')
X_test = process('data/Test.csv')
y = X.target
X = X.set_index('ward').drop('target', axis='columns')
X_test = X_test.set_index('ward')

In [52]:
from sklearn.model_selection import KFold, StratifiedKFold

y_stratified = pd.cut(y, bins=10, labels=False)

skf = StratifiedKFold(n_splits=5, 
                      shuffle=True, 
                      random_state=0)

cv_strategy = list(skf.split(X, y_stratified))

In [26]:
from skopt.space import Real, Categorical, Integer
reg = CatBoostRegressor(verbose = False)
regression_loss_funcs = ['RMSE', 'Tweedie:variance_power=1.5', 
                         'Tweedie:variance_power=1.75', 'Tweedie:variance_power=1.25',
                         'Tweedie:variance_power=1.10', 'Tweedie:variance_power=1.9']

search_spaces = {
    'loss_function': Categorical(regression_loss_funcs),
    'iterations': Integer(10, 2000),
    'depth': Integer(1, 12),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'random_strength': Real(1e-9, 10, 'log-uniform'), # randomness for scoring splits
    'bagging_temperature': Real(0.0, 1.0), # settings of the Bayesian bootstrap
    'l2_leaf_reg': Integer(2, 100), # L2 regularization
   }

In [29]:
# Wrapping everything up into the Bayesian optimizer
from skopt import BayesSearchCV
from skopt.callbacks import DeltaYStopper, DeadlineStopper
opt = BayesSearchCV(estimator=reg,                                    
                    search_spaces=search_spaces,                      
                    scoring=scoring,                                  
                    cv=cv_strategy,                                           
                    n_iter=60,                                        # max number of trials
                    n_points=3,                                       # number of hyperparameter sets evaluated at the same time
                    n_jobs=-1,                                        # number of jobs
                    iid=False,                                        # if not iid it optimizes on the cv score
                    return_train_score=False,                         
                    refit=False,                                      
                    optimizer_kwargs={'base_estimator': 'GP'},        # optmizer parameters: we use Gaussian Process (GP)
                    random_state=0)                                   # random state for replicability

In [30]:
# Running the optimizer
overdone_control = DeltaYStopper(delta=0.0001)               # We stop if the gain of the optimization becomes too small
time_limit_control = DeadlineStopper(total_time=60 * 60 * 7) # We impose a time limit (6 hours)

best_params = report_perf(opt, X, y,'CatBoost_regression', 
                          callbacks=[overdone_control, time_limit_control])

CatBoost_regression took 22782.39 seconds,  candidates checked: 30, best CV score: -3.099 ± 0.195
Best parameters:
OrderedDict([('bagging_temperature', 1.0),
             ('depth', 5),
             ('iterations', 2000),
             ('l2_leaf_reg', 2),
             ('learning_rate', 0.03135580117994885),
             ('loss_function', 'Tweedie:variance_power=1.9'),
             ('random_strength', 3.487917002425111e-05)])



In [53]:
reg = CatBoostRegressor(
                        verbose = False,
                        **best_params)

In [54]:
folds = 20
skf = StratifiedKFold(n_splits=folds,
                      shuffle=True, 
                      random_state=0)

predictions = np.zeros(len(X_test))
for k, (train_idx, val_idx) in enumerate(skf.split(X, y_stratified)):
    reg.fit(X.iloc[train_idx, :], y[train_idx])
    val_preds = reg.predict(X.iloc[val_idx, :])
    val_rmse = mean_squared_error(y_true=y[val_idx], y_pred=val_preds, squared=False)
    print(f"Fold {k} RMSE: {val_rmse:0.5f}")
    predictions += reg.predict(X_test).ravel()
    
predictions /= folds

Fold 0 RMSE: 2.84932
Fold 1 RMSE: 3.65063
Fold 2 RMSE: 2.97240
Fold 3 RMSE: 3.15510
Fold 4 RMSE: 3.13642
Fold 5 RMSE: 3.64197
Fold 6 RMSE: 3.15404
Fold 7 RMSE: 3.11319
Fold 8 RMSE: 2.86668
Fold 9 RMSE: 2.67677
Fold 10 RMSE: 2.81860
Fold 11 RMSE: 2.82681
Fold 12 RMSE: 2.74273
Fold 13 RMSE: 3.26541
Fold 14 RMSE: 2.94281
Fold 15 RMSE: 3.26602
Fold 16 RMSE: 3.23520
Fold 17 RMSE: 3.02163
Fold 18 RMSE: 3.29375
Fold 19 RMSE: 2.78528


In [55]:
def plot_feature_importance(importance,names,model_type):
    
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    #Define size of bar plot
    plt.figure(figsize=(12,10))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
data = process('data/Train.csv')
plot_feature_importance(reg.get_feature_importance(),data.drop(columns=['ward','target']).columns.to_list(),'CATBOOST')

#feature_importance = np.array(reg.get_feature_importance())
#feature_names = np.array(data.drop(columns=['ward','target']).columns.to_list())
    
#Create a DataFrame using a Dictionary
#data={'feature_names':feature_names,'feature_importance':feature_importance}
#fi_df = pd.DataFrame(data)
#fi_df.sort_values(by=['feature_importance'],ascending=False)
#fi_df[fi_df.feature_importance<=0.3]['feature_names'].to_list()


<IPython.core.display.Javascript object>

In [56]:
data.head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_05,pw_06,pw_07,pw_08,lat,lon,NL,dist_1,dist_2,Household_Size
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.000000,0.012928,0.0,0.0,-29.682270,24.734743,0.292039,798.770621,502.854905,3.516501
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.019092,0.004131,0.0,0.0,-29.119311,24.757737,3.207775,747.643998,456.067928,3.877587
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.001421,0.040881,0.0,0.0,-29.142276,25.094093,0.000000,729.796714,434.987083,3.025929
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.005011,0.0,0.0,-29.372052,24.942867,2.038778,759.057513,463.723338,3.294194
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.000673,0.017629,0.0,0.0,-29.409381,25.290165,0.000000,743.194403,445.171419,3.846971


In [58]:
ss = pd.read_csv('data/SampleSubmission.csv')
test = process('data/Test.csv')
test.drop(columns=['ward'],inplace=True)
ss['target'] = reg.predict(test)
#version = 'kwazulu&guteng&limpopo_dist_catboost_targetlabeling_top3_feat_comb'
version = 'kwazulu&guteng&catboost_dist1&2only'
ss.to_csv('submissions/'+version+'.csv', index=False)

In [22]:
data= process('data/Train.csv')
X = data.drop(columns=['target','ward'])
y= data[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2,random_state=0)

In [23]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)
print("RMSE", sqrt(mean_squared_error(y_test,xgb_grid.predict(X_test))))

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[19:00:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[19:00:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


0.8897642886109454
{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
RMSE 3.1420352039665707


In [ ]:
from sklearn.decomposition import PCA
data = process('data/Train.csv')

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(data.drop(columns=['ward','target']))
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, data[['ward','target']]], axis = 1)


In [ ]:
import mpld3

mpld3.enable_notebook()
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize=15)
ax.set_ylabel('Principal Component 2', fontsize=15)
ax.set_title('2 component PCA', fontsize=20)


ax.scatter(finalDf['principal component 1'],
           finalDf['principal component 2'], c=finalDf['target'], s=50)

ax.grid()



In [ ]:
from sklearn.cluster import KMeans
data= process('data/Train.csv')
no_labels = data.drop(columns=['ward','target'])
labels = data[['ward']]
kmeans = KMeans(n_clusters=4)
kmeans.fit(no_labels)
data['cluster'] = pd.Series(kmeans.labels_, index=data.index)

In [ ]:
info_df = data.groupby('cluster').mean()
info_df

In [ ]:
distortions = []
data= process('data/Train.csv')
data = data.drop(columns=['target'])
no_labels = data.drop(columns=['ward'])

K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(no_labels)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
"""combinations=list(itertools.combinations(['ProvinceName','LocalMunicipalityName','DistrictMunicipalityName'], 2))

    combined_features = []

    for (cat1, cat2) in combinations:
        ColName = cat1 + '&' + cat2
        combined.loc[:,ColName] = combined.loc[:,cat1]+'&'+combined.loc[:,cat2]
        combined_features.append(ColName)


    
    for feature in combined_features:
        combined = target_profiling(combined,feature)
        combined = combined.drop(columns=[feature])"""

    """encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(
        combined[['ProvinceName']]).toarray())
    encoder_df.columns = encoder.get_feature_names(['ProvinceName'])

    combined = combined.join(encoder_df)"""

In [ ]:
#uncomment if this is the first time running the notebook
"""
url = 'https://services7.arcgis.com/oeoyTUJC8HEeYsRB/arcgis/rest/services/MDB_Wards_2016/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
response = requests.get(url)
with open('data/mdb_data.json', 'w') as outfile:
    outfile.write(str(json.dumps(response.json())))
"""

In [4]:
ACCESS_KEY = '1a2febdd5530824a8bbc342c32aee3db'
provinces = [
    'Eastern Cape',
    'Free State',
    'Gauteng',
    'KwaZulu-Natal',
    'Limpopo',
    'Mpumalanga',
    'Northern Cape',
    'North West',
    'Western Cape',
]
prov_list=[]
for province in provinces:
    prov_dict={}
    url = 'http://api.positionstack.com/v1/forward?access_key=' + ACCESS_KEY+'&query='+province
    response = requests.get(url)
    prov_dict['province']=province
    prov_dict['lon']=response.json()['data'][0]['longitude']
    prov_dict['lat']=response.json()['data'][0]['latitude']
    prov_list.append(prov_dict)
print(prov_list)


[{'province': 'Eastern Cape', 'lon': 26.501313, 'lat': -32.19228}, {'province': 'Free State', 'lon': 26.682325, 'lat': -28.59211}, {'province': 'Gauteng', 'lon': 28.059764, 'lat': -26.238909}, {'province': 'KwaZulu-Natal', 'lon': 30.717503, 'lat': -28.453644}, {'province': 'Limpopo', 'lon': 29.341886, 'lat': -23.786037}, {'province': 'Mpumalanga', 'lon': 29.854636, 'lat': -26.375424}, {'province': 'Northern Cape', 'lon': 21.846696, 'lat': -29.46391}, {'province': 'North West', 'lon': 25.477234, 'lat': -26.701793}, {'province': 'Western Cape', 'lon': 19.760643, 'lat': -33.582933}]
